In [ ]:
import itertools as it
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import copy
from collections import defaultdict, Counter

In [ ]:
def T_pqr(p,q,r):
    G = nx.Graph()
    if p>2 and q>2 and r >2:
        for i in range(p+q+r-2):
            G.add_node(i, waga=i)
        for i in range (0, p-1):
            G.add_edge(i,i+1)
        for i in range (p,p+q-2):
            G.add_edge(i,i+1)
            G.add_edge(0,p)
        for i in range(p+q-1,p+q+r-3):
            G.add_edge(i,i+1)
            G.add_edge(0,p+q-1)
    if p == 1 or q ==1 or r == 1:
        for i in range(p+q+r-2):
            G.add_node(i, waga=i)
        for i in range (0, p+q+r-3):
            G.add_edge(i,i+1)
    if p == 2:
        for i in range(0,p+q+r-2):
            G.add_node(i, waga=i)
        for i in range (0, 1):
            G.add_edge(i,i+1)
        for i in range (2,q):
            G.add_edge(i,i+1)
            G.add_edge(0,2)
        for i in range(q+1,q+r-1):
            G.add_edge(i,i+1)
            G.add_edge(0,q+1)
    if q ==2 :
        for i in range(0,p+q+r-2):
            G.add_node(i, waga=i)
        for i in range (0, p-1):
            G.add_edge(i,i+1)
            G.add_edge(0,p)
        for i in range(p+2,p+q+r-3):
            G.add_edge(i,i+1)
            G.add_edge(0,p+2)
    if r == 2:
        for i in range(0,p+q+r-2):
            G.add_node(i, waga=i)
        for i in range (0, p-1):
            G.add_edge(i,i+1)
        for i in range (p,p+q-2):
            G.add_edge(i,i+1)
            G.add_edge(0,p)
        G.add_edge(0,p+q-1)
    return G
def symetryczna(A, k):

    max_len = max(len(sublist) for sublist in A)
    zewn = []

    for kombinacja in it.combinations_with_replacement(range(len(A)), k):
        summed_sublist = []
        for j in range(max_len):
            sublist_sum = sum(A[i][j] if j < len(A[i]) else 0 for i in kombinacja)
            summed_sublist.append(sublist_sum)
        zewn.append(summed_sublist)
    return zewn

def zewnetrzna(A, k):

    max_len = max(len(sublist) for sublist in A)
    zewn = []

    for kombinacja in it.combinations(range(len(A)), k):
        summed_sublist = []
        for j in range(max_len):
            sublist_sum = sum(A[i][j] if j < len(A[i]) else 0 for i in kombinacja)
            summed_sublist.append(sublist_sum)
        zewn.append(summed_sublist)
    return zewn
#liczy zewenętrznąale numeruje poszczególne linijki outputu. Dla testów
# czy liczy odpowiednio
def zewnetrzna_num(A, k):

    max_len = max(len(sublist) for sublist in A)
    zewn = []

    for kombinacja in it.combinations(range(len(A)), k):
        summed_sublist = []
        for j in range(max_len):
            sublist_sum = sum(A[i][j] if j < len(A[i]) else 0 for i in kombinacja)
            summed_sublist.append(sublist_sum)
        zewn.append(summed_sublist)
    for idx, wynik in enumerate(zewn, 1):
        print(f"{idx} : {wynik}")

# redefiniowanie znanych funkcji, aby operowały na krotkach wag a nie na listach wag
def nadawanie_wag(G, wagi_lista):
    wagi = []
    for idx, (node, weight) in enumerate(zip(G.nodes, wagi_lista)):
        G.nodes[node]['weight'] = weight
    for _, w in G.nodes(data='weight'):
        wagi.append(w)
    return wagi

def odbicie(G, node, wagi):
    #jeśli chcesz sprawdzić, czy odbicie działa oraz chcesz mieć wypisane wagi, komenda poniżej to umożliwia:
    #print(nadawanie_wag(G, wagi_lista))

    wagi = []

    neighbors = list(G.neighbors(node))

    nowa_waga1 = - G.nodes[node]['weight']
    G.nodes[node]['weight'] = nowa_waga1

    for neighbor in neighbors:
        nowa_waga = G.nodes[neighbor]['weight'] -  G.nodes[node]['weight']
        G.nodes[neighbor]['weight'] = nowa_waga
    for _, w in G.nodes(data='weight'):
        wagi.append(w)
    return wagi
def ro_1(G, wagi):
    wagi = []
    for node in G.nodes():
        G.nodes[node]['weight'] += 1
        wagi.append( G.nodes[node]['weight'])
    return wagi
def ro_2(G, wagi):
    wagi = []
    for node in G.nodes():
        G.nodes[node]['weight'] -= 1
        wagi.append( G.nodes[node]['weight'])
    return wagi

# **Ta część notatnika poniżej dotyczy części opracji ro dopóki wagi są niedodatnie albo waga się powtórzyła z wagą z inputu**

najpierw program, który outputuje wszystkie outputy z podziałem na ujemne i dodatnie orbity

In [ ]:
#podział na plusy i minusy

### aby zwiększyć limit na councie popraw liczbę w linijce:
### while not pozytywne(wagi_kopia) and count < 10000000: (13 linijka, nie licząc pustych)

def rownolegle_mod(G, lista_wag):
    def pozytywne(wagi):
        return all(weight >= 0 for weight in wagi)

    wyniki_dla_wszystkich_list = []

    for wagi in lista_wag:
        wagi_kopia = wagi.copy()
        nadawanie_wag(G, wagi_kopia)

        count = 0
        if pozytywne(wagi_kopia):
            wyniki_dla_wszystkich_list.append((tuple(wagi_kopia), count))
            continue

        poczatkowy_stan = tuple(wagi_kopia)

        while not pozytywne(wagi_kopia) and count < 10000:
            sorted_nodes = sorted(G.nodes(data=True), key=lambda x: x[1]["weight"],
                reverse=True)
            nodes_ujemne = [node for node, data in sorted_nodes
                if isinstance(data.get('weight'), (int, float)) and data['weight'] < 0]

            for node in nodes_ujemne:
                wynik = ro_1(G, wagi_kopia)
                wynik2 = odbicie(G, node, wynik)
                wynik3 = ro_2(G, wynik2)

                wagi_kopia = wynik3.copy()
                count += 1

                if pozytywne(wagi_kopia):
                    wyniki_dla_wszystkich_list.append((tuple(wagi_kopia), count))
                    break

                if tuple(wagi_kopia) == poczatkowy_stan:
                    wyniki_dla_wszystkich_list.append(([], count))
                    break
            else:
                continue
            break

    wystapienia_wag = Counter(waga for waga, _ in wyniki_dla_wszystkich_list if waga)

    waga_do_count = defaultdict(lambda: {'nieparzyste': 0, 'parzyste': 0})

    for waga, count in wyniki_dla_wszystkich_list:
        if waga:
            if wystapienia_wag[waga] == 1 and count == 0:
                waga_do_count[waga]['parzyste'] = 1
                waga_do_count[waga]['nieparzyste'] = '0'

            else:
                if count % 2 == 0:
                    waga_do_count[waga]['parzyste'] += 1
                else:
                    waga_do_count[waga]['nieparzyste'] += 1

    wynik = [
        {'suma_count_nieparzyste': counts['nieparzyste'], 'suma_count_parzyste':counts['parzyste'], 'waga': list(waga)}
        for waga, counts in waga_do_count.items()
    ]

    return wynik

Ten program outputuje tylko wagi, które mają różne iteracje odatnie i ujemne

In [ ]:
#podział na plusy i minusy

### aby zwiększyć limit na councie popraw liczbę w linijce:
### while not pozytywne(wagi_kopia) and count < 10000000: (13 linijka, nie licząc pustych)

### ta procedura redukuje wagi, które mają tyle samo plusów co minusów
def rownolegle(G, lista_wag):
    def pozytywne(wagi):
        return all(weight >= 0 for weight in wagi)

    wyniki_dla_wszystkich_list = []

    for wagi in lista_wag:
        wagi_kopia = wagi.copy()
        nadawanie_wag(G, wagi_kopia)

        count = 0
        if pozytywne(wagi_kopia):
            wyniki_dla_wszystkich_list.append((tuple(wagi_kopia), count))
            continue

        poczatkowy_stan = tuple(wagi_kopia)

        while not pozytywne(wagi_kopia) and count < 10000:
            sorted_nodes = sorted(G.nodes(data=True), key=lambda x: x[1]["weight"],
                reverse=True)
            nodes_ujemne = [node for node, data in sorted_nodes
                if isinstance(data.get('weight'), (int, float)) and data['weight'] < 0]

            for node in nodes_ujemne:
                wynik = ro_1(G, wagi_kopia)
                wynik2 = odbicie(G, node, wynik)
                wynik3 = ro_2(G, wynik2)

                wagi_kopia = wynik3.copy()
                count += 1

                if pozytywne(wagi_kopia):
                    wyniki_dla_wszystkich_list.append((tuple(wagi_kopia), count))
                    break

                if tuple(wagi_kopia) == poczatkowy_stan:
                    wyniki_dla_wszystkich_list.append(([], count))
                    break
            else:
                continue
            break

    wystapienia_wag = Counter(waga for waga, _ in wyniki_dla_wszystkich_list if waga)

    waga_do_count = defaultdict(lambda: {'nieparzyste': 0, 'parzyste': 0})

    for waga, count in wyniki_dla_wszystkich_list:
        if waga:
            if wystapienia_wag.get(waga, 0) == 1 and count == 0:
                waga_do_count[waga]['parzyste'] = 1
                waga_do_count[waga]['nieparzyste'] = 0
            else:
                if count % 2 == 0:
                    waga_do_count[waga]['parzyste'] += 1
                else:
                    waga_do_count[waga]['nieparzyste'] += 1
    wynik = [
        {'suma_count_nieparzyste': counts['nieparzyste'],
         'suma_count_parzyste': counts['parzyste'],
         'waga': waga}
        for waga, counts in waga_do_count.items()
        if counts['parzyste'] != counts['nieparzyste']
    ]
    return wynik

In [42]:
waga = [[0,0,0,0,1],[1,0,0,0,-1],[-1,1,0,1,0],[0,1,0,-1,0],
            [0,-1,1,1,0],[1,-1,1,-1,0],[0,0,-1,1,0],
            [-1,0,1,0,1],[1,0,-1,-1,0],[0,0,0,-1,0],[-1,0,0,1,0],[1,-1,0,0,-1],
           [0,-1,0,0,1],[0,1,-1,0,-1],[0,0,1,0,-1], [-1,1,-1,0,1]]
wagas = symetryczna(waga,5)
wagaz = zewnetrzna(waga,5)

In [ ]:
rownolegle(T_pqr(3,2,2),wagaz)

[{'suma_count_nieparzyste': 0,
  'suma_count_parzyste': 1,
  'waga': (0, 1, 1, 1, 0)},
 {'suma_count_nieparzyste': 0,
  'suma_count_parzyste': 1,
  'waga': (0, 0, 0, 3, 0)}]

# Positive- manipulacja na jednej liście wag tak, aby posprawdzać czy jest jakaś nieujemna. Program ma ustalony limit operacji odbicie, działa na posortowanych wierzchołkach, co powoduje uniknięcie identycznościowych odbić. Program w sensowny sposób sprawdza czy jest w skończonym czasie seria odbić prowadząca do nieujemnej wagi

Ten program odbija w najmniejszym wedle wagi wierzchołku

In [22]:
def positive(G, wagi_lista):
    def pozytywne(G):
        for node, data in G.nodes(data=True):
            if data['weight'] is not None and data['weight'] < 0:
                return False
        return True

    count = 0
    nodes_z_odbiciem = []
    wagi = []

    nadawanie_wag(G, wagi_lista)

    if pozytywne(G):
      return wagi_lista, count

    while count < 10000:
        sorted_nodes = sorted(G.nodes(data=True), key=lambda x: x[1]["weight"], reverse=True)
        nodes_ujemne = [
            node for node, data in sorted_nodes
            if isinstance(data.get('weight'), (int, float)) and data['weight'] < 0]
        for node in nodes_ujemne:
            wynik = ro_1(G, wagi_lista)
            wynik2 = odbicie(G, node, wynik)
            wynik3 = ro_2(G, wynik2)
            nodes_z_odbiciem.append(node)

            count += 1

        if pozytywne(G):
            break

    for _, w in G.nodes(data='weight'):
        wagi.append(w)


    return(wagi, count)

In [23]:
positive(T_pqr(3,2,2), [2,0,0,0,-2])

([1, 0, 0, 0, 0], 1)

Ten program odbija w największym wierzchołku

In [20]:
def positive(G, wagi_lista):
    def pozytywne(G):
        for node, data in G.nodes(data=True):
            if data['weight'] is not None and data['weight'] < 0:
                return False
        return True

    count = 0
    nodes_z_odbiciem = []
    wagi = []

    nadawanie_wag(G, wagi_lista)
    if pozytywne(G):
      return wagi_lista, count

    while count < 10000:
        sorted_nodes = sorted(G.nodes(data=True), key=lambda x: x[1]["weight"], reverse=True)
        nodes_dodatnie = [
            node for node, data in sorted_nodes
            if isinstance(data.get('weight'), (int, float)) and data['weight'] > 0]
        for node in nodes_dodatnie:
            wynik = ro_1(G, wagi_lista)
            wynik2 = odbicie(G, node, wynik)
            wynik3 = ro_2(G, wynik2)
            nodes_z_odbiciem.append(node)

            count += 1

        if pozytywne(G):
            break

    for _, w in G.nodes(data='weight'):
        wagi.append(w)


    return(nodes_z_odbiciem , wagi, count)

In [21]:
positive(T_pqr(3,2,2), [2,0,0,0,2])

([2, 0, 0, 0, 2], 0)

# **pushing_down : dodawanie do symetrycznej ustalonej wagi**



In [56]:
waga = [[0,0,0,0,1],[1,0,0,0,-1],[-1,1,0,1,0],[0,1,0,-1,0],
            [0,-1,1,1,0],[1,-1,1,-1,0],[0,0,-1,1,0],
            [-1,0,1,0,1],[1,0,-1,-1,0],[0,0,0,-1,0],[-1,0,0,1,0],[1,-1,0,0,-1],
           [0,-1,0,0,1],[0,1,-1,0,-1],[0,0,1,0,-1], [-1,1,-1,0,1]]
wagas = symetryczna(waga,5)
wagaz = zewnetrzna(waga,5)

In [57]:
def pushing_down(G, wagi_lista, k, lista):
    zewn = list(symetryczna(wagi_lista, k))
    modified_zewn = []

    for waga in zewn:
        modified_zewn.append([x + y for x, y in zip(waga, lista)])

    return modified_zewn

In [58]:
pushing_down(D5, waga, 2, (1,0,0,0,0))

[[1, 0, 0, 0, 2],
 [2, 0, 0, 0, 0],
 [0, 1, 0, 1, 1],
 [1, 1, 0, -1, 1],
 [1, -1, 1, 1, 1],
 [2, -1, 1, -1, 1],
 [1, 0, -1, 1, 1],
 [0, 0, 1, 0, 2],
 [2, 0, -1, -1, 1],
 [1, 0, 0, -1, 1],
 [0, 0, 0, 1, 1],
 [2, -1, 0, 0, 0],
 [1, -1, 0, 0, 2],
 [1, 1, -1, 0, 0],
 [1, 0, 1, 0, 0],
 [0, 1, -1, 0, 2],
 [3, 0, 0, 0, -2],
 [1, 1, 0, 1, -1],
 [2, 1, 0, -1, -1],
 [2, -1, 1, 1, -1],
 [3, -1, 1, -1, -1],
 [2, 0, -1, 1, -1],
 [1, 0, 1, 0, 0],
 [3, 0, -1, -1, -1],
 [2, 0, 0, -1, -1],
 [1, 0, 0, 1, -1],
 [3, -1, 0, 0, -2],
 [2, -1, 0, 0, 0],
 [2, 1, -1, 0, -2],
 [2, 0, 1, 0, -2],
 [1, 1, -1, 0, 0],
 [-1, 2, 0, 2, 0],
 [0, 2, 0, 0, 0],
 [0, 0, 1, 2, 0],
 [1, 0, 1, 0, 0],
 [0, 1, -1, 2, 0],
 [-1, 1, 1, 1, 1],
 [1, 1, -1, 0, 0],
 [0, 1, 0, 0, 0],
 [-1, 1, 0, 2, 0],
 [1, 0, 0, 1, -1],
 [0, 0, 0, 1, 1],
 [0, 2, -1, 1, -1],
 [0, 1, 1, 1, -1],
 [-1, 2, -1, 1, 1],
 [1, 2, 0, -2, 0],
 [1, 0, 1, 0, 0],
 [2, 0, 1, -2, 0],
 [1, 1, -1, 0, 0],
 [0, 1, 1, -1, 1],
 [2, 1, -1, -2, 0],
 [1, 1, 0, -2, 0],
 [0, 1, 0,